## Part 1: Preprocessing

In [182]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [183]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [184]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [185]:
# Create a list of at least 10 column names to use as X data
x_columns = ['Education',
             'Age',
             'DistanceFromHome',
             'JobSatisfaction',
             'OverTime',
             'StockOptionLevel',
             'WorkLifeBalance',
             'YearsAtCompany',
             'YearsSinceLastPromotion',
             'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[x_columns]

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [186]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [187]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

label_encoder = LabelEncoder()
X_train['OverTime'] = label_encoder.fit_transform(X_train['OverTime'])
X_test['OverTime'] = label_encoder.transform(X_test['OverTime'])

X_train.nunique()

,0
Education,5
Age,43
DistanceFromHome,29
JobSatisfaction,4
OverTime,2
StockOptionLevel,4
WorkLifeBalance,4
YearsAtCompany,32
YearsSinceLastPromotion,16
NumCompaniesWorked,10


In [188]:
# # Create a StandardScaler
# scaler = StandardScaler()
# # Non-categorical data to scale
# data_to_scale = ['Age',
#                  'DistanceFromHome',
#                  'YearsAtCompany',
#                  'YearsSinceLastPromotion',
#                  'NumCompaniesWorked']

# # Fit the StandardScaler to the training data
# scaler.fit(X_train[data_to_scale])

# # Scale the training and testing data
# X_train[data_to_scale] = scaler.transform(X_train[data_to_scale])
# X_test[data_to_scale] = scaler.transform(X_test[data_to_scale])

# X_train.head()



In [189]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [190]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
y_department_test = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))

# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_attrition_test = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))


## Create, Compile, and Train the Model

In [191]:
# Find the number of columns in the X training data
inputs = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(inputs,), name='input_layer')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [192]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(y_department_train.shape[1], activation='sigmoid', name='department_output')(department_dense)

print(department_output)
print(y_department_train.shape)


<KerasTensor shape=(None, 3), dtype=float32, sparse=False, name=keras_tensor_83>
(1102, 3)


In [193]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train.shape[1], activation='sigmoid', name='attrition_output')(attrition_dense)

print(attrition_output)
print(y_attrition_train.shape)

<KerasTensor shape=(None, 2), dtype=float32, sparse=False, name=keras_tensor_85>
(1102, 2)


In [194]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_51 (Dense)          │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_52 (Dense)          │ (None, 128)            │          8,320 │ dense_51[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_53 (Dense)          │ (None, 32)             │          4,128 │ dense_52[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_54 (Dense)          │ (None, 32)             │          4,128 │ dense_52[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ dense_53[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ dense_54[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [195]:
# Train the model
model.fit(
    X_train_scaled,
    {
     'attrition_output': y_department_train,
     'department_output': y_attrition_train
    },
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - attrition_output_accuracy: 0.1792 - attrition_output_loss: 0.5564 - department_output_accuracy: 0.1146 - department_output_loss: 0.6176 - loss: 1.1744 - val_attrition_output_accuracy: 0.0543 - val_attrition_output_loss: 0.4997 - val_department_output_accuracy: 0.1719 - val_department_output_loss: 0.5154 - val_loss: 1.0158
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.0450 - attrition_output_loss: 0.3709 - department_output_accuracy: 0.1372 - department_output_loss: 0.4985 - loss: 0.8697 - val_attrition_output_accuracy: 0.0724 - val_attrition_output_loss: 0.4588 - val_department_output_accuracy: 0.1719 - val_department_output_loss: 0.5144 - val_loss: 0.9736
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.0593 - attrition_output_loss: 0.3561 - department_output_accuracy: 0.1527 - department_output_loss: 0.4856 - loss: 0.8415 - val_attrition_output_accuracy: 0.1041 - va

In [196]:
# Evaluate the model with the testing data


In [197]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.